This program iterates Electrolyser Model v3 and records CAPEX and OPEX for each component parameter combination

In [1]:
#ElectrolyserOptimiser
Version: 3.1

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("C:/Users/tangl/OneDrive/Engineering Course/Year 3/3YP/Data/ninja_weather_51.7508_-1.2538_uncorrected.csv")
class solar_model:
    """Non-dispatchable asset base class"""
    def __init__(self):
        self.capacity = 144 #kW
        self.efficiency = 0.17 #taken from average, will be specified
        self.area = 1.9404 #m^2, taken from 72 cell 99*196cm panel
#class for pvAsset
class pvAsset(solar_model):
    """PV Asset Class"""
    def __init__(self, Power):                              #Power is in MW
        self.capacity = 300
        self.efficiency = 0.183
        self.area = 1.6368
        self.nPanels = round((Power*1000000)/300)
        self.Power = Power
        
class electrolyzer:
    def __init__(self,production_rate,max_power):
        self.production_rate = production_rate     #kg of hydrogen produced per second per MW of electricity
        self.max_power = max_power                 #in MW
        
class alkaline(electrolyzer):
    def __init__(self,production_rate, max_power):
        super().__init__(production_rate, max_power)
        self.min_power = max_power*15/100  # in MW, minimum power to retain its ability to ramp up and down quickly  
        self.lifetime = 65000
        self.install_cost = max_power * 908 *1000
        
class ITM_PEM(electrolyzer):
    def __init__(self,production_rate, max_power):
        super().__init__(production_rate, max_power)
        self.min_power = max_power*10/100  # in MW, minimum power to retain its ability to ramp up and down quickly  
        self.lifetime = 50000
        self.install_cost = max_power * 432 *1000
    
class location:
    def __init__(self, var_waterprice, fixed_waterprice,var_elec_price,fixed_elec_price):
        self.var_waterprice = var_waterprice               # price per m3
        self.fixed_waterprice = fixed_waterprice           # price per day 
        self.var_elec_price = var_elec_price                       #GBP Per MWh
        self.fixed_elec_price = fixed_elec_price                  #GBP Per day
        
class battery:
    def __init__(self, capacity):                           # input capacity is in MWh
        self.power_rating = 0.25 * capacity           # power rating (in MW) of battery is pegged to capacity (in MWh)
        self.capacity = capacity *3600                    # energy capacity of battery in MJ

Oxford = location(1.4570,(17.84/365),136.4, 0.2740)


state = np.ones((8760,1)) 
for i in range(0,105):
    state[0+i*24:7+i*24] =0
    state[17+i*24:25+i*24] =0

for i in range(105,245):
    state[0+i*24:5+i*24] =0
    state[19+i*24:25+i*24] =0

for i in range(245,365):
    state[0+i*24:7+i*24] =0
    state[17+i*24:25+i*24] =0

In [8]:
results = np.zeros((5,496))
trial = 0
for ElectrolyserPower in range(42,43):
    for SolarFarmSize in range (0, 151,10):
        for BatteryCapacity in range (0,31):
            results[0,trial] = ElectrolyserPower
            SolarFarmSizeActual = SolarFarmSize
            results[1,trial] = SolarFarmSizeActual
            results[2,trial] = BatteryCapacity

            #SolarFarmSize = 30                    # MW
            #ElectrolyserPower = 10              # MW
            #BatteryCapacity = 1                 # MWh
            totalIrr = 0.001*df["radiation_surface"] 
            pvChosen = pvAsset(SolarFarmSizeActual)                                     
            percAreaEff = 0.7 #due to angles of tilt vs sunlight etc
            total_capacity = pvChosen.nPanels*pvChosen.capacity
            total_Area = pvChosen.nPanels*pvChosen.area*percAreaEff
            areaEff = total_Area*pvChosen.efficiency
            hourlyGen1 = areaEff*totalIrr

            Power = hourlyGen1*0.001
            T = len(Power)                           # no. of hourly periods
            electrolyser = ITM_PEM(0.00486, ElectrolyserPower)
            batt = battery(BatteryCapacity)

            grid_demand = np.zeros((T,1))            # power required from grid 
            H_produced = np.zeros((T,1))             # kg of hydrogen produced in the hour period
            batt_power = np.zeros((T,1))             #Amt of power loaded into battery in MW
            batt_soc = np.zeros((T,1))               #Amt of energy stored in battery in MJ
            excess_solar = np.zeros((T,1))
            electrolyzer_power = Power
            # Records hydrogen produced in each hour period
            for j in range(T):
                #if electrolyser is on
                if state[j] == 1:
                    H_produced[j] = electrolyser.max_power*electrolyser.production_rate*60*60 
                    if electrolyzer_power[j] >= electrolyser.max_power:                              #if theoretical power exceeds max power
                        batt_power[j] = min((electrolyzer_power[j] - electrolyser.max_power), batt.power_rating)
                        if batt_power[j] == batt.power_rating:
                            excess_solar[j] = electrolyzer_power[j] - batt.power_rating - electrolyser.max_power
                            batt_soc[j] = min((batt_soc[j-1] + batt_power[j]*60*60), batt.capacity)
                        if batt_soc[j-1] == batt.capacity:
                            excess_solar[j] = electrolyzer_power[j] - electrolyser.max_power
                    else:                                                                           #if theoretical power is below min power
                        grid_demand[j] = electrolyser.max_power - electrolyzer_power[j] 
                        if grid_demand[j] > batt.power_rating:                                 #if power demand exceeds battery power rating
                            if batt_soc[j-1] > batt.power_rating*60*60:
                                batt_soc[j] = batt_soc[j-1] - batt.power_rating*3600
                                grid_demand[j] = grid_demand[j] - batt.power_rating
                            else:
                                batt_soc[j] = batt_soc[j-1]
                        elif grid_demand[j] < batt.power_rating:                               #if power demand is within battery power rating
                    #if there is sufficient energy in battery storage
                            if batt_soc[j-1] > grid_demand[j]*60*60:
                                batt_soc[j] = batt_soc[j-1] - grid_demand[j]*3600
                                grid_demand[j] = 0
                            else:
                                batt_soc[j] = batt_soc[j-1]
                
    #if electrolyser is off
                else:
                    H_produced[j] = 0
                
                    if batt_soc[j-1] == batt.capacity:
                        excess_solar[j] = electrolyzer_power[j]
                        batt_soc[j] = batt_soc[j-1]
                    else:
                        if electrolyzer_power[j] > batt.power_rating:
                            excess_solar[j] = electrolyzer_power[j] - batt.power_rating
                            batt_soc[j] = min((batt_soc[j-1] + batt.power_rating*60*60), batt.capacity)
                        else:
                            batt_soc[j] = min((batt_soc[j-1] + electrolyzer_power[j]*60*60), batt.capacity)

                    
            total_annual_hydrogen = sum(H_produced)
            Total_elec_cost =sum(grid_demand* Oxford.var_elec_price) + T/24*Oxford.fixed_elec_price
            Water_consumption = 9                                         #kg of water needed for each kg of hydrogen
            Total_water_price = T/24*Oxford.fixed_waterprice + sum(H_produced*Water_consumption*Oxford.var_waterprice/1000)
            #lifetime = electrolyser.lifetime
            # Net Present Value of electrolyser
            #discount_rate = 0.035
            #operating_hours = np.count_nonzero(H_produced)                 # Number of operating hours in a year
            #years_lifetime = lifetime/operating_hours                      # Lifetime in number of years
            #years_lifetime_rounded = np.round(years_lifetime)              # round off number of years
            #years_lifetime_rounded = int(years_lifetime_rounded)
            #annual_cost = electrolyser.install_cost/years_lifetime_rounded
            #cost = np.zeros((years_lifetime_rounded,1))   
            #for j in range(years_lifetime_rounded):
             #   cost[j] = annual_cost *((1-discount_rate)**j)
            #npv = sum(cost)
            solarfarm_cost = pvChosen.Power * 450000                   # Cost of solar farm in GBP 
            solarfarm_OM = 6700*pvChosen.Power
            batt_cost = batt.capacity/3600 * 89.28                       # Cost of battery in GBP
            Total_Capital_Cost = batt_cost + solarfarm_cost + electrolyser.install_cost
            Total_operating_cost =  solarfarm_OM + Total_water_price + Total_elec_cost - sum(excess_solar)*55.7
            
            results[3,trial] = Total_Capital_Cost
            results[4,trial] = Total_operating_cost
            trial = trial + 1




In [62]:
print('Total hydrogen produced (in kg) is')
print(total_annual_hydrogen)
print('Total co2 emissions (in kg) from electricity is')
print(solar_co2_emissions+grid_co2_emissions)
print('Total cost of grid electricity consumption is')
print(Total_elec_cost)
print('Total cost of water consumption is')
print(Total_water_price)
print('Total solar energy sold(in MWh) is')
print(sum(excess_solar))
print('Total operating lifetime(in hours) is')
print(lifetime)
print('Total operating lifetime(in years) is')
print(years_lifetime_rounded)
print('Electrolyser cost is ')
print(electrolyser.install_cost)
print('Net Present Value of Electrolyser Cost is ')
print(npv)
print('Cost of Solar Farm Installation')
print(solarfarm_cost)
print('Cost of Battery')
print(batt_cost)
print('Total Immediate Cost is')
print(Total_Cost)



Total hydrogen produced (in kg) is
[2000535.62956977]
Total co2 emissions (in kg) from electricity is
[1144660.56222608]
Total cost of grid electricity consumption is
[609989.46581109]
Total cost of water consumption is
[26250.86371055]
Total solar energy sold(in MWh) is
[0.]
Total operating lifetime(in hours) is
50000
Total operating lifetime(in years) is
10
Electrolyser cost is 
14350000
Net Present Value of Electrolyser Cost is 
[12288426.7597223]
Cost of Solar Farm Installation
21000000
Cost of Battery
89.19
Total Immediate Cost is
[35986329.51952164]


In [65]:
print(results)

[[1.50000000e+01 1.50000000e+01 1.50000000e+01 1.50000000e+01
  2.00000000e+01 2.00000000e+01 2.00000000e+01 2.00000000e+01
  2.50000000e+01 2.50000000e+01 2.50000000e+01 2.50000000e+01]
 [2.00000000e+01 2.00000000e+01 2.10000000e+01 2.10000000e+01
  2.00000000e+01 2.00000000e+01 2.10000000e+01 2.10000000e+01
  2.00000000e+01 2.00000000e+01 2.10000000e+01 2.10000000e+01]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00]
 [1.72564790e+06 1.72564790e+06 1.79851750e+06 1.79851750e+06
  1.82295614e+06 1.82295614e+06 1.89357620e+06 1.89357620e+06
  1.93229679e+06 1.93229679e+06 2.00053563e+06 2.00053563e+06]
 [2.89368866e+07 2.89339409e+07 2.99321483e+07 2.99292026e+07
  3.19598869e+07 3.19567366e+07 3.29516116e+07 3.29483931e+07
  3.50018061e+07 3.49983148e+07 3.59897867e+07 3.59863295e+07]]


In [9]:
pd.DataFrame(results).to_csv(r'C:/Users/tangl/OneDrive/Engineering Course/Year 3/3YP/data.csv',index = False)

8760